In [1]:
#geohash
#latitude, longtitude -> hashvalue
class GeoHash:
    def encode(self, latitude, longitude, precision):
        _base32 = "0123456789bcdefghjkmnpqrstuvwxyz"
        lat_bin = self.get_bin(latitude, -90, 90)
        lng_bin = self.get_bin(longitude, -180, 180)
        
        hash_code, b = '', ''
        for i in range(30):
            b += lng_bin[i] + lat_bin[i]

        for i in range(0, 60, 5):
            hash_code += _base32[int(b[i:i + 5], 2)]

        return hash_code[:precision]
            

    def get_bin(self, value, left, right):
        b = ''
        for i in range(30):
            mid = (left + right) / 2.0
            if value > mid:
                left = mid
                b += '1'
            else:
                right = mid
                b += '0'
        return b

In [7]:
#geohash II
#hashvalue -> latitude, longtitude
class GeoHash:
    def decode(self, geohash):
        _base32 = "0123456789bcdefghjkmnpqrstuvwxyz"
        b = ""
        for c in geohash:
            tmp = '{0:05b}'.format(_base32.find(c))#1 conver to binary
            b += tmp
        
        odd = ''.join([b[i] for i in range(0, len(b), 2)])#2 seperate
        even = ''.join([b[i] for i in range(1, len(b), 2)])
        
        location = []
        location.append(self.get_location(-90.0, 90.0, even))
        location.append(self.get_location(-180.0, 180.0, odd))
        return location
    
    def get_location(self, start, end, string):
        for c in string:
            mid = (start + end) / 2.0
            if c == '1':
                start = mid
            else:
                end = mid
        return (start + end) / 2.0

slt = GeoHash()
slt.decode("wx4g0s")


[39.92706298828125, 116.3946533203125]